In [9]:
import sys
sys.path.append("..")

# 求解器类型
from PWPS.Config import CP_SAT_SOLVER, LP_SOLVER, SCIP_SOLVER
# 求解状态
from PWPS.Config import OPTIMAL, FEASIBLE, INFEASIBLE, NOT_SOLVED
# 求解器类
from PWPS.Solver import Solver

求解以下线性规划模型:
<!-- $$
\min \quad 3a + 2\sum_{i=1}^{10}x_i + 5b
$$ --> 

<div align="center"><img style="background: white;" src="https://render.githubusercontent.com/render/math?math=%5Cmin%20%5Cquad%203a%20%2B%202%5Csum_%7Bi%3D1%7D%5E%7B10%7Dx_i%20%2B%205b"></div>

<!-- $$
\textrm{s.t.} \quad 3 * a + b - 10 + \sum_{i=1}^{10}x_i = 0
$$ --> 

<div align="center"><img style="background: white;" src="https://render.githubusercontent.com/render/math?math=%5Ctextrm%7Bs.t.%7D%20%5Cquad%203%20*%20a%20%2B%20b%20-%2010%20%2B%20%5Csum_%7Bi%3D1%7D%5E%7B10%7Dx_i%20%3D%200"></div>
<!-- $$
x_1 \ge x_2
$$ --> 

<div align="center"><img style="background: white;" src="https://render.githubusercontent.com/render/math?math=x_1%20%5Cge%20x_2"></div>

<!-- $$
x_1 = x_2 - 1
$$ --> 

<div align="center"><img style="background: white;" src="https://render.githubusercontent.com/render/math?math=x_1%20%3D%20x_2%20-%201"></div>

<!-- $$
a,b,x_i \in \left\{ 0, 1 \right\}
$$ --> 

<div align="center"><img style="background: white;" src="https://render.githubusercontent.com/render/math?math=a%2Cb%2Cx_i%20%5Cin%20%5Cleft%5C%7B%200%2C%201%20%5Cright%5C%7D"></div>

$x_1$、$x_2$是$01$变量，但是第三项约束要求$x_1=x_2-1$（也即是$x_1 \in \left\{ -1,0 \right\}$），所以第三项约束会导致此问题无解。此时可以调用```CP_SAT_SOLVER```求解器找出冲突的约束项。

In [10]:
# 定义函数或者问题类构建模型
def build_model(solver: Solver):
    all_vars = []
    # 创建决策变量、添加约束及目标函数
    a = solver.new_bool_var("a")
    b = solver.new_bool_var("b")
    x = [solver.new_bool_var(f"x{i}") for i in range(10)]
    all_vars.append(a)
    all_vars.append(b)
    all_vars.extend(x)
    sum_x = sum([item for item in x])
    # 添加约束
    con_1 = 3 * a + b - 10 + sum_x == 0
    con_2 = x[0] >= x[1]
    con_3 = x[0] == x[1] - 1
    solver.add_constraint(con_1, name = "constraint 1")
    solver.add_constraint(con_2, name = "constraint 2")
    solver.add_constraint(con_3, name = "constraint 3")
    # 添加目标函数
    solver.set_obj(3, a)
    for item_x in x:
        solver.set_obj(2, item_x)
    solver.set_obj(5, b)
    return all_vars


In [11]:
solver = Solver(solver_name=LP_SOLVER)
all_vars = build_model(solver=solver)
status = solver.solve()
status

'infeasible'

此时求解器返回求解状态为```'infeasible'```，再调用```CP_SAT_SOLVER```求解器找出相互冲突的约束。当然，因为这里采用的是免费的```ortools```求解器，所以可能计算效率很慢，可能会花费很长时间都没有任何返回，很像程序卡死了，所以建议使用多线程的方式计算冲突约束，到时间就强制kill线程，并给出适当的提示。

In [12]:
if status == INFEASIBLE:
    solver = Solver(solver_name=CP_SAT_SOLVER, compute_IIS=True)
    # solver.compute_IIS = True
    all_vars = build_model(solver=solver)
    conflict_constraints = solver.find_conflict_constraints()
    print(conflict_constraints)

['_ASSUMPTION_constraint 3']
